In [13]:
import regex
import numpy as np
import math

with open('data.txt') as file:
    data = file.read().split('\n\n')

from tqdm import tqdm

In [14]:
def get_monkeys():
    monkeys = []

    for m in data:
        lines = m.splitlines()

        monkey = {}

        monkey['nr'] = int(lines[0].split()[-1].replace(':', ''))
        monkey['items'] =  [int(a) for a in lines[1].split(': ')[-1].split(',')]

        monkey['operation'] = lines[2].split(': ')[-1].split('= ')[-1]

        monkey['test'] = int(lines[3].split('by ')[-1])

        monkey['true'] = int(lines[4].split(' ')[-1])
        monkey['false'] = int(lines[5].split(' ')[-1])

        monkey['inspections'] = 0

        monkeys.append(monkey)

    return monkeys


In [15]:
rounds = 20
monkeys = get_monkeys()

for n in range(rounds * len(monkeys)):
   
    monkey = monkeys[n % len(monkeys)]

    for item in monkey['items']:
        old = item
        new = eval(monkey['operation'])
        
        new = new // 3

        if new % monkey['test'] == 0:
            monkeys[monkey['true']]['items'].append(new)
        else:
            monkeys[monkey['false']]['items'].append(new)
        
        monkey['inspections'] +=1

    monkey['items'] = []

a, b = sorted([m['inspections'] for m in monkeys])[-2:]
a*b

78678

In [16]:
cache = {}

def calculate_sign(monkeys, n):

    old = n
    sign = []

    for monkey in monkeys:
        
        new = eval(monkey['operation'])
        sign.append(new % monkey['test'])

    return tuple(sign)

def calc(monkeys, n):
    old = n
    sign = []

    for monkey in monkeys:
        
        new = eval(monkey['operation'])
        sign.append(calculate_sign(monkeys, new))

    return tuple(sign)

def reduce_number(monkeys, n):

    sign = calc(monkeys, n)
    
    cached = cache.get(sign, None)

    if cached is None:
        cache[sign] = n
        return n
    elif cached < n:
        return cached
    else:
        cache[sign] = n
        return n

rounds = 10000
monkeys = get_monkeys()

for n in tqdm(range(rounds * len(monkeys))):
   
    monkey = monkeys[n % len(monkeys)]

    for item in monkey['items']:
        old = item
        new = eval(monkey['operation'])
        new = reduce_number(monkeys, new)
        

        if new % monkey['test'] == 0:
            monkeys[monkey['true']]['items'].append(new)
        else:
            monkeys[monkey['false']]['items'].append(new)
        
        monkey['inspections'] +=1

    monkey['items'] = []

100%|██████████| 80000/80000 [04:16<00:00, 311.54it/s]


[123834, 112260, 118063, 123821, 11603, 118089, 123818, 55724]

In [17]:
a, b = sorted([m['inspections'] for m in monkeys])[-2:]
a*b

15333249714